# Compile intensitydata.pckl files into one file from all experiments

In [21]:
from pathlib import Path
import numpy as np
import tables
import pandas as pd
import ccModules3 as cc3
from multiprocessing import Pool
import tqdm

In [22]:
parameters_file = 'Project_info.ods'

In [23]:
#get target folder path
target_folder = Path.cwd().parent
print(target_folder)


/media/bushey/202010/JData/C/C07/C07_Data1


In [24]:
parameters = cc3.read_spread( str(target_folder / parameters_file), index_col=0, header=0)

In [25]:
#parameters needed
outputfolder = str(target_folder / parameters['File'].loc['OutPutFolder'])
outputfile = str(Path(outputfolder) / parameters['File'].loc['IntensityFile'])
experi_file = str(target_folder / parameters['File'].loc['Compiled Experiments'])

In [26]:
#getting list of experiments
experi = cc3.read_spread(experi_file, header=0)

In [27]:
experi

,No.,Sample Name,Genotype,Date,Note1,Stage,Stage Start,Media,Starvation start,Rearing Temp,...,Lamina Present,Saline,Baseline Shift,Sample Movement,Responds,Random,Grade,Notes2.1,Cross,path
0,1.0,flya_20220302_C07-4,w;Gr64f-Gal4; Gr64f-Gal4/TM3,20220302,see responses during LED and odor,Adult,2022-02-23,1:500 Retinal Cornmeal,2022-02-28,25,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-4,/media/bushey/202010/JData/C/C07/C07_Data1/202...
1,NaN,flyb_20220302_C07-4,w;Gr64f-Gal4; Gr64f-Gal4/TM3,20220302,LED was not plugged in no stim light,NaN,2022-02-23,1:500 Retinal Cornmeal,2022-02-28,25,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,did not draw rois,C07-4,/media/bushey/202010/JData/C/C07/C07_Data1/202...
2,1.0,flya_20220303_C07-3,SS87269,20220303,Had to restart at first MCH exposure because s...,Adult,2022-02-24,1:500 Retinal Cornmeal,2022-03-02,25,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-3,/media/bushey/202010/JData/C/C07/C07_Data1/202...
3,NaN,flyb_20220303_C07-3,SS87269,20220303,NaN,Adult,2022-02-24,1:500 Retinal Cornmeal,2022-03-02,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3,/media/bushey/202010/JData/C/C07/C07_Data1/202...
4,1.0,flya_20220308_C07-3,SS87269,20220308,NaN,Adult,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-3,/media/bushey/202010/JData/C/C07/C07_Data1/202...
5,NaN,flyb_20220308_C07-3,SS87269,20220308,can see alpha1 but gamma4 beta2 obscured,Adult,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3,/media/bushey/202010/JData/C/C07/C07_Data1/202...
6,NaN,flyc_20220308_C07-3,SS87269,20220308,NaN,NaN,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3,/media/bushey/202010/JData/C/C07/C07_Data1/202...


In [29]:
#for each animal find the intensity_data files and save to column in summary pandas dataframe
intensity = {}
for row, dseries in experi.iterrows():
    intensity_data_files = [str(i) for i in Path(dseries['path']).glob('**/*IntensityData.pckl')]
    with Pool(4) as p:
        output1=list(tqdm.tqdm(p.imap(pd.read_pickle, intensity_data_files), total=len(intensity_data_files)))
    intensity[row] =pd.DataFrame(index=intensity_data_files, data=output1, columns=['sensor'])
experi['intensity'] = pd.Series(intensity)
#output1 = dview.map(ccModules.compileTimeSeriesData, listexceldata)

100%|█████████████████████████████████████████| 15/15 [00:00<00:00, 1016.57it/s]
/home/bushey/.local/share/virtualenvs/CircuitCatcher2-Pfwy5qBR/lib/python3.10/site-packages/pandas/core/internals/construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 16/16 [00:00<00:00, 1175.14it/s]
/home/bushey/.local/share/virtualenvs/CircuitCatcher2-Pfwy5qBR/lib/python3.10/site-packages/pandas/core/internals/construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=ob

In [33]:
#save pickle file
if not Path(outputfolder).is_dir():
    Path(outputfolder).mkdir()
experi.to_pickle(outputfile)